вариант 2, 3

Вариант 2
Цель: Контроль пространственных размерностей через свертки и пулинг.

Спроектируйте сеть для выхода (64, 16, 16) за не более 3 слоев свертки.
Условие: Используйте как минимум один слой с padding=0, чтобы часть пространственной информации была потеряна. Можно менять в таких условиях kernel_size и stride.
Эксперимент: Сравните выход с использованием padding=0 и padding=1, оцените разницу в размерности и среднее значение активаций.

In [1]:
import torch
import torch.nn as nn

torch.manual_seed(0)

x = torch.randn(8, 3, 32, 32)

def stats(t, name="tensor"):
    return {"name": name, "shape": tuple(t.shape), "mean": float(t.mean().item())}

In [2]:
class ModelPad0(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=2, stride=2, padding=0)  # 32→16
        self.act1  = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # 16→16
        self.act2  = nn.ReLU(inplace=True)
    def forward(self, x):
        x = self.act1(self.conv1(x))
        x = self.act2(self.conv2(x))
        return x

In [3]:
class ModelPad1(nn.Module):
    def __init__(self):
        super().__init__()
        # p=1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1)  # 32→16
        self.act1  = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # 16→16
        self.act2  = nn.ReLU(inplace=True)
    def forward(self, x):
        x = self.act1(self.conv1(x))
        x = self.act2(self.conv2(x))
        return x

In [4]:
m0, m1 = ModelPad0(), ModelPad1()
with torch.no_grad():
    y0 = m0(x)
    y1 = m1(x)
print(stats(y0, "pad0_out"))
print(stats(y1, "pad1_out"))

{'name': 'pad0_out', 'shape': (8, 64, 16, 16), 'mean': 0.08679946511983871}
{'name': 'pad1_out', 'shape': (8, 64, 16, 16), 'mean': 0.07252626866102219}


Среднее значение активаций различается,при padding=0 свёртка теряет часть информации на границах, из-за чего усреднённый уровень активации чуть выше (меньше подавления).

Вариант 3. Контроль количества параметров
Цель: Создание компактной сети.

Создайте архитектуру, используя не более 50 000 параметров.
Условие: Один слой должен быть 1x1 сверткой, чтобы уменьшить число каналов.
Эксперимент: Подсчитайте количество параметров каждого слоя и убедитесь, что общая сумма не превышает лимита.

In [5]:
class CompactNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1   = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu1   = nn.ReLU(inplace=True)
        self.conv1x1 = nn.Conv2d(32, 12, kernel_size=1, stride=1, padding=0)  # 1×1 bottleneck
        self.relu2   = nn.ReLU(inplace=True)
        self.conv3   = nn.Conv2d(12, 32, kernel_size=3, stride=1, padding=1)
        self.relu3   = nn.ReLU(inplace=True)
    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv1x1(x))
        x = self.relu3(self.conv3(x))
        return x

In [6]:
model = CompactNet()
with torch.no_grad():
    y = model(x)
print("Output shape:", tuple(y.shape))

Output shape: (8, 32, 32, 32)


In [7]:
from collections import OrderedDict

def layer_params_dict(net: nn.Module):
    out = OrderedDict()
    for name, module in net.named_modules():
        # Учитываем только слои с собственными параметрами (leaf modules)
        has_own_params = any(p.requires_grad for p in module.parameters(recurse=False))
        if name != "" and has_own_params:
            out[name] = sum(p.numel() for p in module.parameters() if p.requires_grad)
    return out

per_layer = layer_params_dict(model)
total = sum(per_layer.values())

for k, v in per_layer.items():
    print(f"{k:10s} -> {v:6d} params")
print("-"*28)
print(f"TOTAL        -> {total:6d} params")

conv1      ->    896 params
conv1x1    ->    396 params
conv3      ->   3488 params
----------------------------
TOTAL        ->   4780 params


условиие, что количество (сумма) параметров каждого слоя не превышает лимита выполняется